Based on notebook 6, for further experiments only knn, xgboost and linear svc will be considered.  

Handle class imbalance by selectively removing poorly classified classes. Investigate the impact of individual classes on the classifier's performance and potentially improve the accuracy by removing problematic classes.

#convert reason to numbers  
{'Mau Serviço Prestado': 0,  
'Condições de entrega': 1,   
'Atraso de entrega': 2,   
'Enganos': 3}  

1 and 3 are the problematic classes. Class 3 has less samples than 1 but still has greater recall, so I'm going to try:  
* removing class 1- Condições de Entrega
* removing class 3 - Enganos
* join classes 0,1,3 vs 2  


After removing the class and training a new classifier, I'm going to apply this new classifier on the excluded class and use the predicted labels as new labels to train another classifier. This can be seen as an iterative error analysis approach, where you iteratively refine the classifier by addressing specific errors or problematic cases.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler #fixed import

import nltk
import re
from langdetect import detect
from scipy.sparse import hstack
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize

In [2]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")
df.head()

,date,title,reason,description,zone,narrative_tfidf,narrative_embeddings,narrative_bert,narrative_tfidf_title,narrative_bert_title,...,tfidf_title,title_tfidf_nolemma,events,keywords,locs,embeddings_title,tfidf_keywords,embeddings_keywords,events_clean,events_tfidf
0,2021-10-11 12:26:32,CTT - Encomenda entregue danificada e com etiq...,0,Bom dia venho por este meio apresentar uma rec...,Lisboa,bom dia venho meio apresentar reclamação ctt e...,bom dia venho por este meio apresentar uma rec...,Bom dia venho por este meio apresentar uma rec...,ctt encomenda entregar danificar etiqueta envi...,ctt - encomenda entregue danificada e com etiq...,...,ctt encomenda entregar danificar etiqueta envi...,ctt encomenda entregue danificada etiquetas en...,"['venho', 'apresentar', 'Enviei', 'destinado',...",apresentar uma reclamação tinha caixa figura e...,"['Israel', 'Aboboda', 'Israel', 'de Braga', 'O...",ctt - encomenda entregue danificada e com etiq...,apresentar reclamação caixa figura envio israe...,apresentar uma reclamação tinha caixa figura e...,venho apresentar enviei destinado feito proteg...,quase destruir figura partida chegar Caixa che...
1,2021-08-31 23:20:15,CTT - Carta com certificado rasgado!,1,- Boa Noite!\r\n\r\n- Venho por este meio recl...,Coimbra,bom noite venho meio reclamar data ás durante ...,- boa noite!\r\n\r\n- venho por este meio recl...,- Boa Noite! Venho por este meio reclamar que ...,ctt carta certificar rasgar bom noite venho me...,ctt - carta com certificado rasgado! - Boa No...,...,ctt carta certificar rasgar,ctt carta certificado rasgado,"['Venho', 'reclamar', 'deparei -me', 'continha...",Lilia Pimentel certificado profissional passag...,['Boa Noite!'],ctt - carta com certificado rasgado !,lilia pimentel certificar profissional passage...,lilia pimentel certificado profissional passag...,venho reclamar deparei -me continha tem,NaN
2,2021-02-01 19:08:32,CTT - Encomenda registada que nunca chegou ao ...,2,"Boa tarde,\r\nNo dia 17 de Dezembro de 2020 en...",Aveiro,bom tarde dia dezembro enviar encomenda França...,"boa tarde,\r\nno dia 00 de dezembro de 0000 en...","Boa tarde, No dia 17 de Dezembro de 2020 envie...",ctt encomenda registar nunca chegar destine o ...,ctt - encomenda registada que nunca chegou ao ...,...,ctt encomenda registar nunca chegar destine o,ctt encomenda registada nunca chegou destino,"['enviei', 'registado', 'vejo', 'chegou', 'sai...",França em correio registado França em correio ...,"['França', 'Roissy', 'ROISSY']",ctt - encomenda registada que nunca chegou ao ...,frança correio registar frança correio encomen...,frança em correio registado frança em correio ...,enviei registado vejo chegou saiu chegou tem p...,resolr
3,2018-04-19 09:04:34,CTT - Correio Verde,2,"Bom dia,\n\nPretendia esclarecer uma situação ...",Lisboa,bom dia pretender esclarecer situação ocorrer ...,"bom dia,\n\npretendia esclarecer uma situação ...","Bom dia, Pretendia esclarecer uma situação que...",ctt correio verde bom dia pretender esclarecer...,"ctt - correio verde Bom dia, Pretendia esclar...",...,ctt correio verde,ctt correio verde,"['Pretendia esclarecer', 'ocorreu', 'desloquei...",Pretendia esclarecer uma situação Pretendia es...,['Lisboa'],ctt - correio verde,pretender esclarecer situação pretender esclar...,pretendia esclarecer uma situação pretendia es...,pretendia esclarecer ocorreu desloquei dado fe...,esclarecer saber semana enviar eventualmente a...
4,2019-07-04 10:20:41,"CTT - Aviso de recepção da carta registada, pr...",2,venho por este meio solicitar a vossa ajuda pa...,Porto,venho meio solicitar vosso ajudar saber onde e...,venho por este meio solicitar a vossa ajuda pa...,venho por este meio solicitar a vossa ajuda pa...,ctt aviso recepção carta registar preciso urge...,"ctt - aviso de recepção da carta registada, pr...",...,ctt aviso recepção carta registar preciso urge...,ctt aviso recepção carta registada preciso urg...,"['venho', 'solicitar', 'saber onde se encontra...",papais para o centro 

In [3]:
df['reason'].unique()

array([0, 1, 2, 3], dtype=int64)

# Remove class 3- enganos

In [4]:
def remove_class_and_train_classifier(dataset, class_to_remove, clf):
    # Filter out the class to remove
    filtered_df = dataset[dataset['reason'] != class_to_remove]
    
    # Separate features and labels
    y = filtered_df['reason']
    X = filtered_df['narrative_tfidf_title']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # Split the remaining data into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)
    
    # Map the class labels to match the expected classes
    # because of error message, the expected classes are [0, 1, 2], but the unique values in y_train are [0, 2, 3]
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)
    
    vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.95, min_df=2)
    X_train_tf = vectorizer.fit_transform(X_train)
    X_val_tf = vectorizer.transform(X_val)
    clf.fit(X_train_tf, y_train_encoded)
    y_val_pred_encoded = clf.predict(X_val_tf)
    
    # Reverse the label encoding to get the original class labels
    y_val_pred = label_encoder.inverse_transform(y_val_pred_encoded)
    
    l=[0,1,2,3]
    l.remove(class_to_remove)
    confusion_mat = confusion_matrix(y_true=y_val, y_pred=y_val_pred, labels=l)
    class_rep= classification_report(y_val, y_val_pred)
    
    print(class_rep)
    print(confusion_mat)
    
    # Create a DataFrame with the original texts, predicted labels, and true labels
    results_df = pd.DataFrame({'narrative_bert_title': dataset.loc[y_val.index, 'narrative_bert_title'],
                               'narrative_tfidf_title': dataset.loc[y_val.index, 'narrative_tfidf_title'],
                               'predicted label': y_val_pred,
                               'true label': y_val})

    misclassified_texts = results_df[results_df['predicted label'] != results_df['true label']]
    
    # Apply the trained classifier on the excluded class
    X_excluded = dataset[dataset['reason'] == class_to_remove]['narrative_tfidf_title']
    X_excluded_tf = vectorizer.transform(X_excluded)
    y_excluded_pred_encoded = clf.predict(X_excluded_tf)
    y_excluded_pred = label_encoder.inverse_transform(y_excluded_pred_encoded)

    excluded_predictions_df = pd.DataFrame({'narrative_bert_title': dataset.loc[dataset['reason'] == class_to_remove, 'narrative_bert_title'],
                                            'narrative_tfidf_title': dataset.loc[dataset['reason'] == class_to_remove, 'narrative_tfidf_title'],
                                            'reason': y_excluded_pred,
                                            'initial Label': dataset.loc[dataset['reason'] == class_to_remove, 'reason']})
    
    return y_val_pred, y_val, misclassified_texts, excluded_predictions_df, filtered_df

In [5]:
def new_classifier(df, clf, class_to_remove):
    # Separate features and labels
    y = df['reason']
    X = df['narrative_tfidf_title']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # Split the remaining data into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)
    
    # Map the class labels to match the expected classes
    # because of error message, the expected classes are [0, 1, 2], but the unique values in y_train are [0, 2, 3]
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)
    
    vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.95, min_df=2)
    X_train_tf = vectorizer.fit_transform(X_train)
    X_val_tf = vectorizer.transform(X_val)
    clf.fit(X_train_tf, y_train_encoded)
    y_val_pred_encoded = clf.predict(X_val_tf)
    
    # Reverse the label encoding to get the original class labels
    y_val_pred = label_encoder.inverse_transform(y_val_pred_encoded)
    
    l=[0,1,2,3]
    l.remove(class_to_remove)
    confusion_mat = confusion_matrix(y_true=y_val, y_pred=y_val_pred, labels=l)
    class_rep= classification_report(y_val, y_val_pred)
    
    print(class_rep)
    print(confusion_mat)

# KNN

###  Misclassified texts

In [6]:
knn_predictions, knn_true_labels, knn_misclassified_texts, knn_predictions_excluded, knn_filtered_df= remove_class_and_train_classifier(df, 3,KNeighborsClassifier())

              precision    recall  f1-score   support

           0       0.56      0.63      0.59      1195
           1       0.26      0.11      0.16       333
           2       0.72      0.75      0.73      1609

    accuracy                           0.63      3137
   macro avg       0.51      0.49      0.49      3137
weighted avg       0.61      0.63      0.62      3137

[[ 748   72  375]
 [ 203   37   93]
 [ 378   32 1199]]


In [7]:
knn_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
7233,ctt - encomenda registada anda a 15 dias para ...,ctt encomenda registar andar dia entregar desd...,0,2
2620,ctt - previsão de entrega da encomenda Boa ta...,ctt previsão entregar encomenda bom tarde dia ...,0,2
5671,ctt - entregas em edifícios sem campainha Boa...,ctt entrega edifício campainha bom tarde habit...,0,1
18749,ctt - não recebi nem uma carta de nenhuma enco...,ctt recebi carta nenhum encomenda mencionar ba...,2,0
6436,"ctt - encomenda parada nos ctt Boa tarde , ap...",ctt encomenda parada ctt bom tarde apo ter rec...,2,0
...,...,...,...,...
18105,ctt - processo aduaneiro pendente com proposta...,ctt processo aduaneiro pendente propor desalfa...,2,0
22354,ctt - extravio de correio Bom dia. Depois de ...,ctt extravio correio bom dia ter enviar carta ...,1,0
6618,"Boa tarde, no passado dia 09/07/2020 recebi um...",bom tarde passar dia recebi carta abrir entreg...,1,0
11747,ctt - envio à cobrança com status devolvido ap...,ctt envio cobrança status devolver após ter se...,2,0


In [8]:
print('MISCLASSIFIED KNN TEXTS')
for i in range(100):
    print(knn_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(knn_misclassified_texts['true label'].iloc[i],
                                                                knn_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED KNN TEXTS
ctt - encomenda registada anda a 15 dias para ser entregue  estou desde o dia 22 de fevereiro a espera de uma encomenda que me foi enviada, registada pelos CTT. Enviei um mail a pedir informações, com o numero de registo da guia e recebo um mail a indicar que teria de dar uma panóplia de dados (acho que pelo numero de guia, estes deveriam ter essa mesma informação) e para responder ao mail que me enviaram, não mexendo no assunto. Para resolver isto o mais rápido possível pois não tarda faz um mês que solicitei o material. Enviei copia da guia de registo, com os dados e fotografia da caixa da encomenda. Quando recebo um mail de não entregue pois o destinatário não existe. Já decidi que não vou comprar mais nada cuja entrega seja feita por esta péssima distribuidora. Divulgam tantos serviços novos, mas depois não tem nem pessoas suficientes nem serviços compatíveis com os padrões de qualidade de referem ter. PÉSSIMA EMPRESA não recomendo. 
TRUE LABEL IS 2 BUT IT W

### Apply classifier to the excluded class

In [9]:
knn_predictions_excluded

,narrative_bert_title,narrative_tfidf_title,reason,initial Label
14,ctt - encomenda que nem chegou a casa Estive ...,ctt encomenda chegar casa retir casa aguardar ...,0,3
23,ctt - aviso de desalfandegamento não entregue ...,ctt aviso desalfandegamento entregar bom tarde...,0,3
29,ctt - encomenda entregue na morada errada Fiz...,ctt encomenda entregar morada errar fiz encome...,0,3
40,ctt - ninguém me sabe dizer nada sobre a local...,ctt ninguém saber dizer nada sobre localização...,2,3
62,ctt - recebem e não fazem o serviço Paguei o ...,ctt receber fazer serviço paguar envio artigo ...,2,3
...,...,...,...,...
22345,"ctt - encomenda dada como entregue, mas nunca ...",ctt encomenda dar entregar nunca recebir passa...,1,3
22366,ctt - iva cobrado duas vezes Venho por este m...,ctt iva cobrar dois vez venho meio reclamar no...,0,3
22367,ctt - encomenda entregue na localidade errada!...,bom tarde fiz vários enviar dia junho encomend...,2,3
22411,ctt - encomenda recebida fantasma Venho por e...,ctt encomenda recebir fantasma venho meio recl...,0,3


In [10]:
print('PREDICTIONS OF KNN ON EXCLUDED TEXTS')
for i in range(100):
    print(knn_predictions_excluded['narrative_bert_title'].iloc[i])
    print('INITIAL LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(knn_predictions_excluded['initial Label'].iloc[i],
                                                                knn_predictions_excluded['reason'].iloc[i]))
    print('\n')

PREDICTIONS OF KNN ON EXCLUDED TEXTS
ctt - encomenda que nem chegou a casa  Estive retida em casa a aguardar que tocassem à porta visto ter recebido mensagem no dia anterior como iria receber a encomenda. Para meu espanto recebo mensagem a dizer que não estava ninguém em casa. E vou ver o correio e verifico que nem o postal estava na caixa. Estamos a ser burlados pelos os ctts pois não gastam combustível a entregar na morada e temos nós de gastarmos combustível para ir levantar uma encomenda que os mesmos deveria ter entregue pois estive em casa o dia todo. 
INITIAL LABEL IS 3 BUT IT WAS CLASSIFIED AS 0


ctt - aviso de desalfandegamento não entregue  Boa tarde, O meu nome é David Ventura e recentemente através das aplicações de acompanhamento de entregas deparo-me que as encomendas: LX164669215CN LT052844682NL Ficaram suspensas na alfandega. Não deixa de ser o meu espanto, porque acho que não estou a cometer nenhum crime nem acho que estou a cometer algum ato de contrafação, mas já lá

### Use predicted classes as new labels to train new classifier

In [11]:
# Concatenate the excluded predictions dataframe with the original dataset, keeping only the specified columns
new_dataset = pd.concat([knn_filtered_df[['reason', 'narrative_tfidf_title','narrative_bert_title']], 
                         knn_predictions_excluded[['reason', 'narrative_tfidf_title','narrative_bert_title']]], ignore_index=True)


shuffled_dataset = new_dataset.sample(frac=1, random_state=42)
shuffled_dataset

,reason,narrative_tfidf_title,narrative_bert_title
6769,2,ctt encomenda entregar bom tarde enviar encome...,ctt - a minha encomenda não foi entregue Boa ...
8171,2,ctt encomenda recebir aguardo encomenda remeti...,ctt - encomenda não recebida Aguardo uma enco...
6603,0,incompetente ficar assignar entregar encomenda...,O incompetente a quem ficou assignada a entreg...
5305,2,ctt doc ok enviar ad destinatário pedir encome...,"ctt - 2 - doc. não ok, enviado ad ao destinatá..."
20251,0,ctt pagamento contra reembolso exmos srs solic...,ctt - não pagamentos de contra reembolsos Exm...
...,...,...,...
11964,0,ctt entregar internacional estagnar ação marca...,ctt - entrega internacional estagnada e sem aç...
21575,0,ctt falha entregar destinatário bom dia passar...,ctt - falha na entrega ao destinatário Bom di...
5390,2,ctt encomenda entregar dois encomenda distribu...,ctt - encomenda não entregue Tinha duas encom...
860,2,ctt entregar retir alfândegar desde julho poss...,ctt - a entrega está a ser retido na alfândega...


In [12]:
new_classifier(shuffled_dataset, KNeighborsClassifier(), 3)

              precision    recall  f1-score   support

           0       0.59      0.64      0.61      1351
           1       0.26      0.13      0.17       350
           2       0.70      0.74      0.72      1663

    accuracy                           0.63      3364
   macro avg       0.52      0.50      0.50      3364
weighted avg       0.61      0.63      0.62      3364

[[ 861   74  416]
 [ 209   44   97]
 [ 390   49 1224]]


# XGBoost

###  Misclassified texts

In [13]:
xgb_predictions, xgb_true_labels, xgb_misclassified_texts, xgb_predictions_excluded, xgb_filtered_df= remove_class_and_train_classifier(df, 3,XGBClassifier())

              precision    recall  f1-score   support

           0       0.65      0.65      0.65      1195
           1       0.43      0.09      0.15       333
           2       0.75      0.86      0.80      1609

    accuracy                           0.70      3137
   macro avg       0.61      0.54      0.53      3137
weighted avg       0.67      0.70      0.67      3137

[[ 782   36  377]
 [ 206   31   96]
 [ 220    5 1384]]


In [14]:
xgb_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
5671,ctt - entregas em edifícios sem campainha Boa...,ctt entrega edifício campainha bom tarde habit...,0,1
2395,"ctt - correspondência aberta Bom dia, Serve o...",ctt correspondência abrir bom dia servir prese...,0,1
18749,ctt - não recebi nem uma carta de nenhuma enco...,ctt recebi carta nenhum encomenda mencionar ba...,2,0
6436,"ctt - encomenda parada nos ctt Boa tarde , ap...",ctt encomenda parada ctt bom tarde apo ter rec...,2,0
4357,ctt - encomenda não chegou Fiz um envio para ...,ctt encomenda chegar fiz envio Brasil julho en...,0,2
...,...,...,...,...
8631,"Olá, é inadmissível que uma encomenda contra-r...",olá inadmissível encomenda enviar dia recebir ...,0,2
8359,ctt - falta consecutiva e frequente de entrega...,ctt falta consecutivo frequente entregar corre...,0,2
15651,"RECEBI UM AVISO PARA DESALFANDEGAMENTO, ENVIEI...",recebi aviso desalfandegamento enviar resposta...,2,0
6618,"Boa tarde, no passado dia 09/07/2020 recebi um...",bom tarde passar dia recebi carta abrir entreg...,2,0


In [15]:
print('MISCLASSIFIED XGB TEXTS')
for i in range(100):
    print(xgb_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(xgb_misclassified_texts['true label'].iloc[i],
                                                                xgb_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED XGB TEXTS
ctt - entregas em edifícios sem campainha  Boa tarde, Habito num prédio de 3 andares cujo acesso às escadas é livre e desempedido (não existe qualquer porta, qualquer pessoa pode aceder a qualquer andar, não existe elevador e não existem campainhas no piso térreo). Desde sempre o carteiro (que por curiosidade se desloca de mota) recusa-se a entregar qualquer encomenda à porta, sob o protesto de que o prédio tem que ter campainhas e que ele não é obrigado a subir. No balcão mais próximo dos CTT, dizem que o carteiro vem da central e que a informação que têm é a mesma, que o prédio tem que ter campainhas e que ele não é obrigado a subir à porta. Confrontei o carteiro que não se quis identificar quando o solicitei. Gostaria que esta informação fosse validada: - Como posso reclamar de um individuo dos CTT que se recusa a indentificar? - É verdade que ele não é obrigado a entregar à porta? (Caso afirmativo, pretendo que me seja disponibilizado acesso ao regulamento o

### Apply classifier to the excluded class

In [16]:
xgb_predictions_excluded

,narrative_bert_title,narrative_tfidf_title,reason,initial Label
14,ctt - encomenda que nem chegou a casa Estive ...,ctt encomenda chegar casa retir casa aguardar ...,0,3
23,ctt - aviso de desalfandegamento não entregue ...,ctt aviso desalfandegamento entregar bom tarde...,0,3
29,ctt - encomenda entregue na morada errada Fiz...,ctt encomenda entregar morada errar fiz encome...,0,3
40,ctt - ninguém me sabe dizer nada sobre a local...,ctt ninguém saber dizer nada sobre localização...,2,3
62,ctt - recebem e não fazem o serviço Paguei o ...,ctt receber fazer serviço paguar envio artigo ...,0,3
...,...,...,...,...
22345,"ctt - encomenda dada como entregue, mas nunca ...",ctt encomenda dar entregar nunca recebir passa...,0,3
22366,ctt - iva cobrado duas vezes Venho por este m...,ctt iva cobrar dois vez venho meio reclamar no...,0,3
22367,ctt - encomenda entregue na localidade errada!...,bom tarde fiz vários enviar dia junho encomend...,0,3
22411,ctt - encomenda recebida fantasma Venho por e...,ctt encomenda recebir fantasma venho meio recl...,0,3


In [17]:
print('PREDICTIONS OF XGB ON EXCLUDED TEXTS')
for i in range(100):
    print(xgb_predictions_excluded['narrative_bert_title'].iloc[i])
    print('INITIAL LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(xgb_predictions_excluded['initial Label'].iloc[i],
                                                                xgb_predictions_excluded['reason'].iloc[i]))
    print('\n')

PREDICTIONS OF XGB ON EXCLUDED TEXTS
ctt - encomenda que nem chegou a casa  Estive retida em casa a aguardar que tocassem à porta visto ter recebido mensagem no dia anterior como iria receber a encomenda. Para meu espanto recebo mensagem a dizer que não estava ninguém em casa. E vou ver o correio e verifico que nem o postal estava na caixa. Estamos a ser burlados pelos os ctts pois não gastam combustível a entregar na morada e temos nós de gastarmos combustível para ir levantar uma encomenda que os mesmos deveria ter entregue pois estive em casa o dia todo. 
INITIAL LABEL IS 3 BUT IT WAS CLASSIFIED AS 0


ctt - aviso de desalfandegamento não entregue  Boa tarde, O meu nome é David Ventura e recentemente através das aplicações de acompanhamento de entregas deparo-me que as encomendas: LX164669215CN LT052844682NL Ficaram suspensas na alfandega. Não deixa de ser o meu espanto, porque acho que não estou a cometer nenhum crime nem acho que estou a cometer algum ato de contrafação, mas já lá

### Use predicted classes as new labels to train new classifier

In [18]:
# Concatenate the excluded predictions dataframe with the original dataset, keeping only the specified columns
new_dataset = pd.concat([xgb_filtered_df[['reason', 'narrative_tfidf_title','narrative_bert_title']], 
                         xgb_predictions_excluded[['reason', 'narrative_tfidf_title','narrative_bert_title']]], ignore_index=True)


shuffled_dataset = new_dataset.sample(frac=1, random_state=42)
shuffled_dataset

,reason,narrative_tfidf_title,narrative_bert_title
6769,2,ctt encomenda entregar bom tarde enviar encome...,ctt - a minha encomenda não foi entregue Boa ...
8171,2,ctt encomenda recebir aguardo encomenda remeti...,ctt - encomenda não recebida Aguardo uma enco...
6603,0,incompetente ficar assignar entregar encomenda...,O incompetente a quem ficou assignada a entreg...
5305,2,ctt doc ok enviar ad destinatário pedir encome...,"ctt - 2 - doc. não ok, enviado ad ao destinatá..."
20251,0,ctt pagamento contra reembolso exmos srs solic...,ctt - não pagamentos de contra reembolsos Exm...
...,...,...,...
11964,0,ctt entregar internacional estagnar ação marca...,ctt - entrega internacional estagnada e sem aç...
21575,0,ctt falha entregar destinatário bom dia passar...,ctt - falha na entrega ao destinatário Bom di...
5390,2,ctt encomenda entregar dois encomenda distribu...,ctt - encomenda não entregue Tinha duas encom...
860,2,ctt entregar retir alfândegar desde julho poss...,ctt - a entrega está a ser retido na alfândega...


In [19]:
new_classifier(shuffled_dataset, XGBClassifier(), 3)

              precision    recall  f1-score   support

           0       0.67      0.70      0.68      1352
           1       0.43      0.08      0.14       349
           2       0.75      0.85      0.80      1663

    accuracy                           0.71      3364
   macro avg       0.62      0.54      0.54      3364
weighted avg       0.68      0.71      0.68      3364

[[ 947   30  375]
 [ 231   29   89]
 [ 245    8 1410]]


# SVC

###  Misclassified texts

In [20]:
svc_predictions, svc_true_labels, svc_misclassified_texts, svc_predictions_excluded, svc_filtered_df= remove_class_and_train_classifier(df, 3,LinearSVC())

              precision    recall  f1-score   support

           0       0.64      0.69      0.66      1195
           1       0.36      0.10      0.15       333
           2       0.76      0.83      0.80      1609

    accuracy                           0.70      3137
   macro avg       0.59      0.54      0.54      3137
weighted avg       0.67      0.70      0.68      3137

[[ 824   40  331]
 [ 214   32   87]
 [ 253   16 1340]]


In [21]:
svc_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
5671,ctt - entregas em edifícios sem campainha Boa...,ctt entrega edifício campainha bom tarde habit...,0,1
2395,"ctt - correspondência aberta Bom dia, Serve o...",ctt correspondência abrir bom dia servir prese...,0,1
6436,"ctt - encomenda parada nos ctt Boa tarde , ap...",ctt encomenda parada ctt bom tarde apo ter rec...,2,0
4357,ctt - encomenda não chegou Fiz um envio para ...,ctt encomenda chegar fiz envio Brasil julho en...,0,2
16831,ctt - carta não entregue há um mês Bom dia en...,ctt carta entregar mês bom dia entregamos dia ...,0,2
...,...,...,...,...
8631,"Olá, é inadmissível que uma encomenda contra-r...",olá inadmissível encomenda enviar dia recebir ...,0,2
8359,ctt - falta consecutiva e frequente de entrega...,ctt falta consecutivo frequente entregar corre...,0,2
15651,"RECEBI UM AVISO PARA DESALFANDEGAMENTO, ENVIEI...",recebi aviso desalfandegamento enviar resposta...,2,0
6618,"Boa tarde, no passado dia 09/07/2020 recebi um...",bom tarde passar dia recebi carta abrir entreg...,2,0


In [22]:
print('MISCLASSIFIED SVC TEXTS')
for i in range(100):
    print(svc_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(svc_misclassified_texts['true label'].iloc[i],
                                                                svc_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED SVC TEXTS
ctt - entregas em edifícios sem campainha  Boa tarde, Habito num prédio de 3 andares cujo acesso às escadas é livre e desempedido (não existe qualquer porta, qualquer pessoa pode aceder a qualquer andar, não existe elevador e não existem campainhas no piso térreo). Desde sempre o carteiro (que por curiosidade se desloca de mota) recusa-se a entregar qualquer encomenda à porta, sob o protesto de que o prédio tem que ter campainhas e que ele não é obrigado a subir. No balcão mais próximo dos CTT, dizem que o carteiro vem da central e que a informação que têm é a mesma, que o prédio tem que ter campainhas e que ele não é obrigado a subir à porta. Confrontei o carteiro que não se quis identificar quando o solicitei. Gostaria que esta informação fosse validada: - Como posso reclamar de um individuo dos CTT que se recusa a indentificar? - É verdade que ele não é obrigado a entregar à porta? (Caso afirmativo, pretendo que me seja disponibilizado acesso ao regulamento o

### Apply classifier to the excluded class

In [23]:
svc_predictions_excluded

,narrative_bert_title,narrative_tfidf_title,reason,initial Label
14,ctt - encomenda que nem chegou a casa Estive ...,ctt encomenda chegar casa retir casa aguardar ...,0,3
23,ctt - aviso de desalfandegamento não entregue ...,ctt aviso desalfandegamento entregar bom tarde...,0,3
29,ctt - encomenda entregue na morada errada Fiz...,ctt encomenda entregar morada errar fiz encome...,0,3
40,ctt - ninguém me sabe dizer nada sobre a local...,ctt ninguém saber dizer nada sobre localização...,2,3
62,ctt - recebem e não fazem o serviço Paguei o ...,ctt receber fazer serviço paguar envio artigo ...,0,3
...,...,...,...,...
22345,"ctt - encomenda dada como entregue, mas nunca ...",ctt encomenda dar entregar nunca recebir passa...,0,3
22366,ctt - iva cobrado duas vezes Venho por este m...,ctt iva cobrar dois vez venho meio reclamar no...,0,3
22367,ctt - encomenda entregue na localidade errada!...,bom tarde fiz vários enviar dia junho encomend...,2,3
22411,ctt - encomenda recebida fantasma Venho por e...,ctt encomenda recebir fantasma venho meio recl...,0,3


In [24]:
print('PREDICTIONS OF SVC ON EXCLUDED TEXTS')
for i in range(100):
    print(svc_predictions_excluded['narrative_bert_title'].iloc[i])
    print('INITIAL LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(svc_predictions_excluded['initial Label'].iloc[i],
                                                                svc_predictions_excluded['reason'].iloc[i]))
    print('\n')

PREDICTIONS OF SVC ON EXCLUDED TEXTS
ctt - encomenda que nem chegou a casa  Estive retida em casa a aguardar que tocassem à porta visto ter recebido mensagem no dia anterior como iria receber a encomenda. Para meu espanto recebo mensagem a dizer que não estava ninguém em casa. E vou ver o correio e verifico que nem o postal estava na caixa. Estamos a ser burlados pelos os ctts pois não gastam combustível a entregar na morada e temos nós de gastarmos combustível para ir levantar uma encomenda que os mesmos deveria ter entregue pois estive em casa o dia todo. 
INITIAL LABEL IS 3 BUT IT WAS CLASSIFIED AS 0


ctt - aviso de desalfandegamento não entregue  Boa tarde, O meu nome é David Ventura e recentemente através das aplicações de acompanhamento de entregas deparo-me que as encomendas: LX164669215CN LT052844682NL Ficaram suspensas na alfandega. Não deixa de ser o meu espanto, porque acho que não estou a cometer nenhum crime nem acho que estou a cometer algum ato de contrafação, mas já lá

### Use predicted classes as new labels to train new classifier

In [25]:
# Concatenate the excluded predictions dataframe with the original dataset, keeping only the specified columns
new_dataset = pd.concat([svc_filtered_df[['reason', 'narrative_tfidf_title','narrative_bert_title']], 
                         svc_predictions_excluded[['reason', 'narrative_tfidf_title','narrative_bert_title']]], ignore_index=True)


shuffled_dataset = new_dataset.sample(frac=1, random_state=42)
shuffled_dataset

,reason,narrative_tfidf_title,narrative_bert_title
6769,2,ctt encomenda entregar bom tarde enviar encome...,ctt - a minha encomenda não foi entregue Boa ...
8171,2,ctt encomenda recebir aguardo encomenda remeti...,ctt - encomenda não recebida Aguardo uma enco...
6603,0,incompetente ficar assignar entregar encomenda...,O incompetente a quem ficou assignada a entreg...
5305,2,ctt doc ok enviar ad destinatário pedir encome...,"ctt - 2 - doc. não ok, enviado ad ao destinatá..."
20251,0,ctt pagamento contra reembolso exmos srs solic...,ctt - não pagamentos de contra reembolsos Exm...
...,...,...,...
11964,0,ctt entregar internacional estagnar ação marca...,ctt - entrega internacional estagnada e sem aç...
21575,0,ctt falha entregar destinatário bom dia passar...,ctt - falha na entrega ao destinatário Bom di...
5390,2,ctt encomenda entregar dois encomenda distribu...,ctt - encomenda não entregue Tinha duas encom...
860,2,ctt entregar retir alfândegar desde julho poss...,ctt - a entrega está a ser retido na alfândega...


In [26]:
new_classifier(shuffled_dataset, LinearSVC(), 3)

              precision    recall  f1-score   support

           0       0.65      0.73      0.69      1358
           1       0.56      0.11      0.19       346
           2       0.76      0.81      0.79      1660

    accuracy                           0.71      3364
   macro avg       0.66      0.55      0.56      3364
weighted avg       0.70      0.71      0.69      3364

[[ 995   24  339]
 [ 223   39   84]
 [ 302    7 1351]]
